In [ ]:
import warnings
warnings.simplefilter("ignore")
warnings.filterwarnings("ignore")

import time, sys
import datetime
import tqdm.notebook as tqdm
import pandas as pd
import numpy as np

import requests
import selenium
from bs4 import BeautifulSoup
from selenium import webdriver
from urllib.parse import urljoin
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as ec
from selenium.common.exceptions import TimeoutException, StaleElementReferenceException
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException

In [ ]:
driver=webdriver.Chrome(r"C:\Drivers\webdriver\\chromedriver.exe")

In [ ]:
url = "https://www.expedia.co.in/Flights"

# Setting the ticket types paths
return_ticket = "//label[@id='flight-type-roundtrip-label-hp-flight']"
one_way_ticket = "//label[@id='flight-type-one-way-label-hp-flight']"
multi_ticket = "//label[@id='flight-type-multi-dest-label-hp-flight']"

In [ ]:
def ticket_chooser(ticket):

    try:
        ticket_type = browser.find_element_by_xpath(ticket)
        ticket_type.click()
    except Exception as e:
        pass

In [ ]:
# For departure place name
def dep_place_chooser(dep_place):
    fly_from = browser.find_element_by_xpath("//input[@id='flight-origin-hp-flight']")
    time.sleep(1)
    fly_from.clear()
    time.sleep(1.5)
    fly_from.send_keys('  ' + dep_place)
    time.sleep(1.5)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1.5)
    first_item.click()

In [ ]:
# For arrival place name
def arr_place_chooser(arr_place):
    fly_to = browser.find_element_by_xpath("//input[@id='flight-destination-hp-flight']")
    time.sleep(1)
    fly_to.clear()
    time.sleep(1.5)
    fly_to.send_keys('  ' + arr_place)
    time.sleep(1.5)
    first_item = browser.find_element_by_xpath("//a[@id='aria-option-0']")
    time.sleep(1.5)
    first_item.click()

In [ ]:
# Departure date
def dep_date_chooser(month, day, year):
    dep_date_button = browser.find_element_by_xpath("//input[@id='flight-departing-hp-flight']")
    dep_date_button.clear()
    dep_date_button.send_keys(month + '/' + day + '/' + year)

In [ ]:
# Return date
def return_date_chooser(month, day, year):
    return_date_button = browser.find_element_by_xpath("//input[@id='flight-returning-hp-flight']")

    for i in range(11):
        return_date_button.send_keys(Keys.BACKSPACE)
    return_date_button.send_keys(month + '/' + day + '/' + year)

In [ ]:
def search():
    search = browser.find_element_by_xpath("//button[@class='btn-primary btn-action gcw-submit']")
    search.click()
    time.sleep(15)
    print('Results ready!')
df = pd.DataFrame()
def compile_data():
    global df
    global dep_times_list
    global arr_times_list
    global airlines_list
    global price_list
    global durations_list
    global stops_list
    global layovers_list

    #departure times
    dep_times = browser.find_elements_by_xpath("//span[@data-test-id='departure-time']")
    dep_times_list = [value.text for value in dep_times]

    #arrival times
    arr_times = browser.find_elements_by_xpath("//span[@data-test-id='arrival-time']")
    arr_times_list = [value.text for value in arr_times]

    #airline name
    airlines = browser.find_elements_by_xpath("//span[@data-test-id='airline-name']")
    airlines_list = [value.text for value in airlines]

    #prices
    prices = browser.find_elements_by_xpath("//span[@data-test-id='listing-price-dollars']")
    price_list = [value.text.split('/div>)[1] for value in prices]

    #durations
    durations = browser.find_elements_by_xpath("//span[@data-test-id='duration']")
    durations_list = [value.text for value in durations]
                                   
    #stops
    stops = browser.find_elements_by_xpath("//span[@class='number-stops']")
    stops_list = [value.text for value in stops]

    #layovers
    layovers = browser.find_elements_by_xpath("//span[@data-test-id='layover-airport-stops']")
    layovers_list = [value.text for value in layovers]

    now = datetime.datetime.now()
    current_date = (str(now.year) + '-' + str(now.month) + '-' + str(now.day))
    current_time = (str(now.hour) + ':' + str(now.minute))
    current_price = 'price' + '(' + current_date + '---' + current_time + ')'
    for i in range(len(dep_times_list)):
        try:
            df.loc[i, 'departure_time'] = dep_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'arrival_time'] = arr_times_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'airline'] = airlines_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'duration'] = durations_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'stops'] = stops_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, 'layovers'] = layovers_list[i]
        except Exception as e:
            pass
        try:
            df.loc[i, str(current_price)] = price_list[i]
       except Exception as e:
            pass

    print('Comma Separated Value File Created!')

In [ ]:
for i in range(8):    
    browser.get(url)
    time.sleep(5)

    # choose flights only
    flights_only = browser.find_element_by_xpath("//button[@id='tab-flight-tab-hp']")
    flights_only.click()

    ticket_chooser(return_ticket)

    dep_place_chooser('BOM')

    arr_place_chooser('BLR')

    dep_date_chooser('04', '01', '2022')

    return_date_chooser('05', '02', '2022')

    search()

    compile_data()

    print('run {} completed!'.format(i))

    df.to_csv('Flight_Price_Data.csv')

    time.sleep(15)                            

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split 
from sklearn.metrics import r2_score, mean_squared_error,mean_absolute_error
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import OrdinalEncoder
from statsmodels.stats.outliers_influence import variance_inflation_factor
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [ ]:
train_data = pd.read_excel(r"E:\capstoneprojects\Datasets\flight_price_prediction\Data_Train.xlsx")
train_data

In [ ]:
test_data = pd.read_excel(r"E:\capstoneprojects\Datasets\flight_price_prediction\Test_set.xlsx")
test_data

In [ ]:
df = pd.concat([test_data, train_data])

In [ ]:
df

In [ ]:
df.describe()

In [ ]:
df.shape

In [ ]:
df.isnull().sum()

In [ ]:
df.isnull().sum().sum()

In [ ]:
df['Route']= df['Route'].fillna(df['Route'].mode()[0])
df['Total_Stops']= df['Total_Stops'].fillna(df['Total_Stops'].mode()[0])
df['Price']= df['Price'].fillna(df['Price'].mean())                                        

In [ ]:
df.isnull().sum().sum()

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(30,8),facecolor='lightgreen')
sns.countplot(x='Airline', data=df)

In [ ]:
##jet airways are more in no.as compared to other flights....other flights which are more frequent other than jet airways are 
#Indigo  and Air india.....

In [ ]:
plt.figure(figsize=(30,8),facecolor='lightgreen')
sns.violinplot(x='Airline', y='Price', data=df,hue='Source')

In [ ]:
#Air india ,Jet airways,spice jet,indigo,vistara MAINLY strts from delhi,kolkata,banglore,mumbai,chennai
#carrier flights only originate from delhi


In [ ]:
plt.figure(figsize=(30,8),facecolor='lightgreen')
sns.stripplot(x='Airline', y='Price', data=df)

In [ ]:
#jet airways have price between 3000 to 20,000 mostly except some exceptional cases
#air india also has price varying between 3000 to 20,000

In [ ]:
plt.figure(figsize=(30,8),facecolor='lightgreen')
sns.stripplot(x='Airline', y='Price',hue='Total_Stops' ,data=df)

In [ ]:
#most jet airways flights have mostly one stops ...
#air india has mostly two stops..
#multiple carriers flight have only one stop...
# some flights of jet airways , indigo,,spicejet,vistara,air india are non stops .....
#very few flights have 3 stops ....
#mostly no flights have 4 stops

In [ ]:
plt.figure(figsize=(30,8),facecolor='lightgreen')
sns.stripplot(x='Airline', y='Price',hue='Source' ,data=df)

In [ ]:
plt.figure(figsize=(30,8),facecolor='lightgreen')
sns.stripplot(x='Airline', y='Price',hue='Destination',data=df)

In [ ]:
#many flights have destination hyderabad and new delhi ,cochin,banglore....
##jet airways & air india flights have all major cities as destiation i.e. cochin,banglore,delhi,new delhi,hyderabad,kolkata
#multiple carriers flights only go to cochin

In [ ]:
df.dtypes

In [ ]:
df["Journey_day"] = pd.to_datetime(df.Date_of_Journey, format="%d/%m/%Y").dt.day
df["Journey_month"] = pd.to_datetime(df["Date_of_Journey"], format = "%d/%m/%Y").dt.month
df.head()

In [ ]:
# So,as we have converted Date_of_Journey column into integers, Now we can drop this column.

df.drop(["Date_of_Journey"], axis = 1, inplace = True)

In [ ]:
# Departure time
df["Dep_hour"] = pd.to_datetime(df["Dep_Time"]).dt.hour
df["Dep_min"] = pd.to_datetime(df["Dep_Time"]).dt.minute

# Now we can drop Dep_Time,
df.drop(["Dep_Time"], axis = 1, inplace = True)
df.head()

In [ ]:
# Arrival time 

df["Arrival_hour"] = pd.to_datetime(df.Arrival_Time).dt.hour
df["Arrival_min"] = pd.to_datetime(df.Arrival_Time).dt.minute

# Now we can drop Arrival_Time ,
df.drop(["Arrival_Time"], axis = 1, inplace = True)
df.head()

In [ ]:
# Assigning and converting Duration column into list
duration = list(df["Duration"])

for i in range(len(duration)):
    if len(duration[i].split()) != 2:   
        if "h" in duration[i]:
            duration[i] = duration[i].strip() + " 0m"   
        else:
            duration[i] = "0h " + duration[i]           

duration_hours = []
duration_mins = []
for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep = "h")[0]))   
    duration_mins.append(int(duration[i].split(sep = "m")[0].split()[-1]))  

In [ ]:
# Adding duration_hours and duration_mins list to df dataframe

df["Duration_hours"] = duration_hours
df["Duration_mins"] = duration_mins
df.drop(["Duration"], axis = 1, inplace = True)
df.head()

In [ ]:
enc=OrdinalEncoder()
for i in df.columns:
    if df[i].dtypes=="object":
        df[i]=enc.fit_transform(df[i].values.reshape(-1,1))
df

In [ ]:
df

In [ ]:
df.skew()

In [ ]:
y = df['Price']
X = df.drop(columns=['Price'])

In [ ]:
np.abs(3)

In [ ]:
np.abs(-3)

In [ ]:
from scipy.stats import zscore
(np.abs(zscore(X))<3).all()

In [ ]:
#Data scaling Z=(X-mean)/std

scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)

In [ ]:
X_scaled.shape[1]

In [ ]:
vif=pd.DataFrame()
vif["vif"]=[variance_inflation_factor(X_scaled,i) for i in range(X_scaled.shape[1])]
vif["Features"]=X.columns
vif

In [ ]:
rf = RandomForestRegressor()
for i in range(0,100):
    X_train,X_test,y_train,y_test =train_test_split(X_scaled,y,test_size=0.2,random_state=i)
    rf.fit(X_train,y_train)
    pred_train = rf.predict(X_train)
    pred_test= rf.predict(X_test)
    print(f"At random state{i},the training accuracy is:-{r2_score(y_train,pred_train)}")
    print(f"At random state{i},the testing accuracy is:-{r2_score(y_test,pred_test)}")
    print("\n")

In [ ]:
#so as we can interpret random state 87 is giving best train and test accuracy,so we train our model at random state 87:

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=87)

In [ ]:
rf=RandomForestRegressor()
rf.fit(X_train,y_train)
rf.score(X_train,y_train)
y_pred=rf.predict(X_test)
rfs=r2_score(y_test,y_pred)
print('R2 score:',rfs*100)

rfscore=cross_val_score(rf,X,y,cv=5)
rfc=rfscore.mean()
print('cross val score:',rfc*100)

In [ ]:
y_pred=rf.predict(X_test)
y_train.head()

In [ ]:
#Adjusted R2 score
rf.score(X_train,y_train)

In [ ]:
rf.score(X_test,y_test)

In [ ]:
#lets plot and visualize
y_pred=rf.predict(X_test)
y_pred

In [ ]:
plt.scatter(y_test,y_pred,color='r')
plt.xlabel='actual PRICE'
plt.ylabel='Predicted PRICE'
plt.title='Actual vs model predicted '
plt.show()

In [ ]:
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
dt=DecisionTreeRegressor()
dt.fit(X_train,y_train)
dt.score(X_train,y_train)
y_pred=dt.predict(X_test)
dtr=r2_score(y_test,y_pred)
print('R2 score:',dtr*100)

dtscore=cross_val_score(dt,X,y,cv=5)
dtr=dtscore.mean()
print('cross val score:',dtr*100)

In [ ]:
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
from sklearn.ensemble import ExtraTreesRegressor

In [ ]:
model = ExtraTreesRegressor()
model.fit(X_train,y_train)
model.score(X_train,y_train)
y_pred=model.predict(X_test)
modelr=r2_score(y_test,y_pred)
print('R2 score:',modelr*100)
modelscore=cross_val_score(model,X,y,cv=5)
modelr=modelscore.mean()
print('cross val score:',modelr*100)

In [ ]:
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
import xgboost as xgb
from xgboost import XGBRegressor
xg=XGBRegressor()

xg.fit(X_train,y_train)
xg.score(X_train,y_train)
y_pred=xg.predict(X_test)
xg_score=r2_score(y_test,y_pred)
print('R2 score:',xg_score*100)
xgscore=cross_val_score(xg,X,y,cv=5)
xgr=dtscore.mean()
print('cross val score:',xgr*100)

In [ ]:
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
from sklearn.ensemble import AdaBoostRegressor
ada=AdaBoostRegressor()
ada.fit(X_train,y_train)
ada.score(X_train,y_train)
y_pred=ada.predict(X_test)
ada_score=r2_score(y_test,y_pred)
print('R2 score:',ada_score*100)

adascore=cross_val_score(ada,X,y,cv=5)
adab=adascore.mean()
print('cross val score:',adab*100)

In [ ]:
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:', mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
#as observed the random forest regressor works well with this dataset...so we hypertune parameters of final model RandomForestRegressor using gridsearchCV

parameters ={'criterion':['mse','mae'],'max_features':['auto','sqrt','log2'],'min_samples_split':[2], 
             'min_samples_leaf':[1], 'bootstrap':[True]}

In [ ]:
rf=RandomForestRegressor()
gcv=GridSearchCV(rf,parameters)
gcv.fit(X_train,y_train)
print(gcv.best_params_)

In [ ]:
rf=RandomForestRegressor(criterion='mse',max_features='log2',min_samples_split= 2 ,
             min_samples_leaf=1, bootstrap=True)
rf.fit(X_train,y_train)
rf.score(X_train,y_train)
y_pred=rf.predict(X_test)
rfs=r2_score(y_test,y_pred)
print('R2 score:',rfs*100)
rfscore=cross_val_score(rf,X,y,cv=5)
rfr=rfscore.mean()
print('cross val score:',rfr*100)

In [ ]:
print('MAE:', mean_absolute_error(y_test, y_pred))
print('MSE:',mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(mean_squared_error(y_test, y_pred)))

In [ ]:
plt.scatter(y_test,y_pred,color='r')
plt.xlabel='actual PRICE'
plt.ylabel='Predicted PRICE'
plt.title='Actual vs model predicted '
plt.show()

In [ ]:
#the best model is RandomForestRegressor with 71% accuracy and 62% cross validation score & 
#MAE: 1541.4725361417416
#MSE: 5051623.706897248
#RMSE: 2247.5817464326515

In [ ]:
#saving the model to local file system
filename='flight price prediction_model.pickle'
pickle.dump(rf,open(filename,'wb'))